In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import csv
import sys
from collections import Counter, defaultdict
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import silhouette_score
from sklearn import preprocessing
from sklearn.cluster import KMeans

In [2]:
url = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/cleaned_players.csv'
Data = pd.read_csv(url)
Data

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type
0,Mesut,Özil,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.6,68,MID
1,Sokratis,Papastathopoulos,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.1,49,DEF
2,David,Luiz Moreira Marinho,0,0,7,364,7,23.1,72.0,22.0,0,52,11.7,0,0,0,0.9,55,DEF
3,Pierre-Emerick,Aubameyang,2,1,35,896,12,163.5,139.6,251.0,1,117,55.4,3,0,2,10.6,116,MID
4,Cédric,Soares,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.2,47,DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,Ki-Jana,Hoever,0,0,2,29,1,15.2,13.8,2.0,0,9,3.1,0,0,0,0.0,45,DEF
599,Nélson,Cabral Semedo,0,0,29,689,7,124.5,136.6,48.0,2,139,31.1,3,0,0,3.5,56,DEF
600,Meritan,Shabani,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,45,MID
601,Rayan,Ait Nouri,1,0,19,255,2,51.4,64.2,41.0,3,62,15.7,1,0,0,0.4,50,DEF


In [4]:
class fantasy_Suggestion:
    def _init_(self):
        self.ClassifiedData = pd.DataFrame()
        

    def fit(self,data):
        MIDdata = data[data["element_type"]== "MID"]
        GKdata = data[data["element_type"]== "GK"]
        DEFdata = data[data["element_type"]== "DEF"]
        FWDdata = data[data["element_type"]== "FWD"]
        
        dtMi = MIDdata[['goals_scored','assists','total_points','minutes','creativity','influence','threat','bonus','bps','ict_index','clean_sheets','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtMid = self.NormalizedTable(dtMi).to_numpy()
        #dtMid = StandardizedTable(dtMi).to_numpy()
        dtGk = GKdata[['goals_scored','assists','total_points','minutes','goals_conceded','influence','bonus','bps','ict_index','clean_sheets','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtGk = self.NormalizedTable(dtGk).to_numpy()
        #dtGk = StandardizedTable(dtG).to_numpy()
        dtDe = DEFdata[['goals_scored','assists','total_points','minutes','goals_conceded','creativity','influence','threat','bonus','bps','ict_index','clean_sheets','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtDef = self.NormalizedTable(dtDe).to_numpy()
        #dtDef = StandardizedTable(dtDe).to_numpy()
        dtFw = FWDdata[['goals_scored','assists','total_points','minutes','creativity','influence','threat','bonus','bps','ict_index','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtFwd = self.NormalizedTable(dtFw).to_numpy()
        #dtFwd = StandardizedTable(dtFw).to_numpy()
        
        
        kmeanMid = K_Means()
        kmeanDef = K_Means()
        kmeanGk = K_Means()
        kmeanFwd = K_Means()
        kmeanMid.fit(dtMid,5)
        kmeanDef.fit(dtDef,5)
        kmeanGk.fit(dtGk,3)
        kmeanFwd.fit(dtFwd,5)
        DefClassify = list(np.asarray(kmeanDef.label)+5)
        GkClassify = list(np.asarray(kmeanGk.label)+10)
        FwdClassify = list(np.asarray(kmeanFwd.label)+13)
        MIDdata = MIDdata.assign(classify = kmeanMid.label)
        DEFdata = DEFdata.assign(classify = DefClassify)
        GKdata = GKdata.assign(classify = GkClassify)
        FWDdata = FWDdata.assign(classify = FwdClassify)
        
        # group large classification into small tier
        
        # Clustering the Mid players
        kmeanMid0 = K_Means()
        Mid0 = self.PlayerClassify(MIDdata,0)
        dtMi0 = Mid0[['goals_scored','assists','total_points','minutes','creativity','influence','threat','bonus','bps','ict_index','clean_sheets','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtMi0 = self.NormalizedTable(dtMi0).to_numpy()
        kmeanMid0.fit(dtMi0,4)
        Mid0 = Mid0.assign(Tier = kmeanMid0.label)
        kmeanMid1 = K_Means()
        Mid1 = self.PlayerClassify(MIDdata,1)
        dtMi1 = Mid1[['goals_scored','assists','total_points','minutes','creativity','influence','threat','bonus','bps','ict_index','clean_sheets','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtMi1 = self.NormalizedTable(dtMi1).to_numpy()
        kmeanMid1.fit(dtMi1,6)
        Mid1 = Mid1.assign(Tier = kmeanMid1.label)
        Mid2 = self.PlayerClassify(MIDdata,2)
        Mid2 = Mid2.assign(Tier = 1)
        Mid3 = self.PlayerClassify(MIDdata,3)
        Mid3 = Mid3.assign(Tier = 1)
        Mid4 = self.PlayerClassify(MIDdata,4)
        Mid4 = Mid4.assign(Tier = 1)
        MIDdata = pd.concat([Mid0, Mid1, Mid2, Mid3, Mid4], ignore_index=True)
        
        #Clustering the Defenders
        kmeanDef0 = K_Means()
        Def0 = self.PlayerClassify(DEFdata,5)
        dtDf0 = Def0[['goals_scored','assists','total_points','minutes','goals_conceded','creativity','influence','threat','bonus','bps','ict_index','clean_sheets','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtDf0 = self.NormalizedTable(dtDf0).to_numpy()
        kmeanDef0.fit(dtDf0,6)
        Def0 = Def0.assign(Tier = kmeanDef0.label)
        kmeanDef1 = K_Means()
        Def1 = self.PlayerClassify(DEFdata,6)
        dtDf1 = Def1[['goals_scored','assists','total_points','minutes','goals_conceded','creativity','influence','threat','bonus','bps','ict_index','clean_sheets','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtDf1 = self.NormalizedTable(dtDf1).to_numpy()
        kmeanDef1.fit(dtDf1,5)
        Def1 = Def1.assign(Tier = kmeanDef1.label)
        Def2 = self.PlayerClassify(DEFdata,7)
        Def2 = Def2.assign(Tier = 1)
        Def3 = self.PlayerClassify(DEFdata,8)
        Def3 = Def3.assign(Tier = 1)
        Def4 = self.PlayerClassify(DEFdata,9)
        Def4 = Def4.assign(Tier = 1)
        DEFdata = pd.concat([Def0, Def1, Def2, Def3, Def4], ignore_index=True)
        
       #No clustering for GoalKeeper
        GKdata = GKdata.assign(Tier = 1)
        
       #Clustering for Forward
        kmeanFwd0 = K_Means()
        Fwd0 = self.PlayerClassify(FWDdata,13)
        dtFw0 = Fwd0[['goals_scored','assists','total_points','minutes','creativity','influence','threat','bonus','bps','ict_index','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtFw0 = self.NormalizedTable(dtFw0).to_numpy()
        kmeanFwd0.fit(dtFw0,4)
        Fwd0 = Fwd0.assign(Tier = kmeanFwd0.label)
        kmeanFwd1 = K_Means()
        Fwd1 = self.PlayerClassify(FWDdata,14)
        dtFw1 = Fwd1[['goals_scored','assists','total_points','minutes','creativity','influence','threat','bonus','bps','ict_index','red_cards','yellow_cards','selected_by_percent','now_cost']]
        dtFw1 = self.NormalizedTable(dtFw1).to_numpy()
        kmeanFwd1.fit(dtFw1,3)
        Fwd1 = Fwd1.assign(Tier = kmeanFwd1.label)
        Fwd2 = self.PlayerClassify(FWDdata,15)
        Fwd2 = Fwd2.assign(Tier = 1)
        Fwd3 = self.PlayerClassify(FWDdata,16)
        Fwd3 = Fwd3.assign(Tier = 1)
        Fwd4 = self.PlayerClassify(FWDdata,17)
        Fwd4 = Fwd4.assign(Tier = 1)
        FWDdata = pd.concat([Fwd0, Fwd1, Fwd2, Fwd3, Fwd4], ignore_index=True)
     
        self.ClassifiedData = pd.concat([MIDdata, DEFdata, GKdata, FWDdata], ignore_index=True)
        return self.ClassifiedData
        
            
    def NormalizedTable(self, data):
        dat = data.values
        min_max_scaler = preprocessing.Normalizer()
        x_scaled = min_max_scaler.fit_transform(dat)
        df = pd.DataFrame(x_scaled)
        return df
    
    def getSimilarPlayer(self,first_name, last_name):
        col = self.ClassifiedData[(self.ClassifiedData["first_name"] == first_name) & (self.ClassifiedData["second_name"]== last_name)]
        label = col["classify"].values
        tier = col["Tier"].values
        return self.ClassifiedData[(self.ClassifiedData["classify"] == label[0]) & (self.ClassifiedData["Tier"] == tier[0])]
    
    def getPlayerTier(self,first_name, last_name):
        col = cls[(kdb["first_name"] == first_name) & (cls["second_name"]== last_name)]
        label = col["Tier"].values
        return cls[cls["Tier"] == label[0]]
    
    def PlayerClassify(self, data, clas):
        col = data[(data["classify"] == clas)]
        return col
    

In [5]:
fan = fantasy_Suggestion()
fans = fan.fit(Data)

fan.getSimilarPlayer("Raheem", "Sterling")

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,...,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type,classify,Tier
61,Pierre-Emerick,Aubameyang,2,1,35,896,12,163.5,139.6,251.0,...,117,55.4,3,0,2,10.6,116,MID,1,1
65,Bukayo,Saka,1,0,21,633,8,124.7,115.8,229.0,...,84,47.0,2,0,0,2.5,52,MID,1,1
72,Mahmoud Ahmed,Ibrahim Hassan,0,1,25,715,10,121.6,111.8,324.0,...,75,55.6,4,0,0,0.9,53,MID,1,1
99,Wilfried,Zaha,5,2,54,720,12,162.8,243.6,346.0,...,144,75.3,1,0,2,16.2,73,MID,1,1
112,Ruben,Loftus-Cheek,1,0,16,328,4,38.7,60.6,96.0,...,55,19.5,2,0,1,0.2,59,MID,1,1
121,Ademola,Lookman,2,1,29,561,8,162.5,175.4,220.0,...,95,55.8,1,0,1,0.5,50,MID,1,1
125,Ayoze,Pérez,0,0,8,280,4,36.8,33.0,103.0,...,23,17.4,1,0,1,0.8,60,MID,1,1
127,Harvey,Barnes,3,1,42,718,11,82.1,178.8,351.0,...,109,61.1,2,0,0,4.1,69,MID,1,1
135,Hélder Wander,Sousa de Azevedo e Costa,2,2,34,623,16,110.4,150.2,190.0,...,94,44.9,2,0,0,5.9,55,MID,1,1
137,Ian Carlo,Poveda-Ocampo,0,0,6,148,2,43.3,26.8,70.0,...,23,14.1,0,0,0,0.4,44,MID,1,1


In [6]:
fan.getSimilarPlayer("Jordan", "Pickford")

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,...,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type,classify,Tier
459,Bernd,Leno,0,0,40,900,12,0.0,225.6,0.0,...,192,22.6,3,0,0,8.8,50,GK,12,1
462,Emiliano,Martínez,0,0,42,810,13,0.0,208.6,0.0,...,189,20.8,4,0,0,26.8,48,GK,12,1
466,Mathew,Ryan,0,0,22,810,14,0.3,90.6,0.0,...,129,9.0,2,0,1,13.1,45,GK,12,1
470,Nick,Pope,0,0,39,720,12,0.0,203.4,0.0,...,156,20.3,3,0,0,9.5,54,GK,12,1
474,Kepa,Arrizabalaga,0,0,6,270,6,0.0,50.8,0.0,...,40,5.0,0,0,0,0.7,48,GK,12,1
475,Edouard,Mendy,0,0,33,540,1,0.0,66.8,0.0,...,132,6.7,5,0,0,7.9,52,GK,12,1
478,Vicente,Guaita,0,0,26,900,15,0.0,208.2,0.0,...,153,20.8,1,0,2,3.8,50,GK,12,1
482,Jordan,Pickford,0,0,23,810,15,10.0,216.0,0.0,...,152,22.7,1,0,0,6.7,49,GK,12,1
488,Alphonse,Areola,0,0,25,810,16,0.0,263.4,0.0,...,159,26.1,1,0,0,1.3,45,GK,12,1
489,Kasper,Schmeichel,0,0,40,900,14,0.0,279.4,0.0,...,187,27.9,3,0,0,11.9,55,GK,12,1


In [8]:
fan.getSimilarPlayer("Thiago", "Thiago")

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,...,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type,classify,Tier
263,Jimmy,Dunne,1,0,8,180,5,21.2,66.2,38.0,...,35,12.6,0,0,0,6.2,40,DEF,5,1
266,Kurt,Zouma,3,0,59,810,7,20.2,267.6,101.0,...,221,38.9,5,0,1,20.4,55,DEF,5,1
268,Thiago,Thiago,1,0,34,522,4,19.8,173.4,52.0,...,147,24.6,4,0,0,3.4,55,DEF,5,1
294,Timothy,Castagne,1,3,35,540,8,78.1,145.0,86.0,...,136,31.0,2,0,1,8.1,57,DEF,5,1
338,Serge,Aurier,1,1,25,360,4,39.5,131.8,68.0,...,104,23.9,2,0,0,1.8,52,DEF,5,1


In [9]:
fan.getSimilarPlayer("Jamie", "Vardy")

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,...,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type,classify,Tier
527,Alexandre,Lacazette,3,0,28,494,5,74.1,126.4,188.0,...,91,38.5,2,0,1,2.8,83,FWD,13,0
529,Ollie,Watkins,6,1,49,810,13,155.3,269.6,366.0,...,178,78.5,4,0,0,9.5,61,FWD,13,0
530,Neal,Maupay,4,1,39,733,13,100.7,187.2,390.0,...,129,67.4,2,0,0,2.9,63,FWD,13,0
532,Danny,Welbeck,1,1,17,281,2,60.3,65.2,129.0,...,63,25.5,1,0,0,0.4,55,FWD,13,0
537,Timo,Werner,4,3,50,843,9,131.0,215.2,376.0,...,189,72.3,5,0,0,23.1,95,FWD,13,0
538,Tammy,Abraham,3,2,38,545,5,106.1,154.2,309.0,...,127,56.6,4,0,0,2.8,72,FWD,13,0
541,Dominic,Calvert-Lewin,10,1,72,878,17,116.2,393.2,540.0,...,288,104.9,1,0,1,61.2,79,FWD,13,0
542,Richarlison,de Andrade,1,4,29,547,8,90.1,142.8,316.0,...,103,54.7,1,1,1,4.4,78,FWD,13,0
543,Bobby,Decordova-Reid,3,1,28,533,8,52.9,135.6,182.0,...,130,36.9,2,0,3,0.2,53,FWD,13,0
544,Aleksandar,Mitrović,2,2,30,692,12,140.4,143.4,339.0,...,79,61.0,1,0,2,5.6,56,FWD,13,0


In [10]:
fan.getSimilarPlayer("Heung-Min", "Son")

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,...,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type,classify,Tier
209,Jack,Grealish,5,6,71,810,13,349.7,364.0,498.0,...,260,121.0,4,0,2,41.9,77,MID,2,1
210,Bertrand,Traoré,0,1,11,155,5,42.4,31.4,71.0,...,37,14.5,0,0,0,0.0,59,MID,2,1
211,Alireza,Jahanbakhsh,0,1,7,54,1,43.5,36.8,63.0,...,36,14.3,0,0,1,0.0,54,MID,2,1
212,Callum,Hudson-Odoi,1,0,12,168,0,72.6,49.6,65.0,...,50,18.7,1,0,0,0.3,57,MID,2,1
213,Sadio,Mané,4,3,50,733,10,192.4,240.0,431.0,...,144,86.5,2,0,2,10.9,121,MID,2,1
214,Mohamed,Salah,8,2,73,782,16,297.1,365.2,555.0,...,231,121.7,2,0,0,29.9,122,MID,2,1
215,Diogo,Jota,5,0,46,482,11,84.2,196.6,266.0,...,135,54.7,1,0,0,26.0,68,MID,2,1
216,Kevin,De Bruyne,1,5,39,652,10,374.0,231.2,288.0,...,152,89.5,2,0,0,20.5,117,MID,2,1
217,Riyad,Mahrez,4,1,45,591,9,259.2,214.4,328.0,...,154,80.2,3,0,0,4.9,83,MID,2,1
218,Bruno Miguel,Borges Fernandes,7,4,74,749,14,420.7,422.0,315.0,...,265,115.9,2,0,2,39.3,109,MID,2,1


In [3]:
class K_Means:
    def __init__(self,tol=0.0001, max_iter=100):
        self.k = 0
        self.tol = tol
        self.max_iter = max_iter
        self.classify = {}
        self.inertia = 0
        self.label = []
    
    def setClassification(self,data):
        self.classify = data
        
    def createLabel(self,row, arr):
        for a in range(len(arr)):
            for element in arr.get(a):
                if np.array_equal(row,element):
                    return a
                
    # function to compute euclidean distance 
    def distance(self,p1, p2): 
        return np.sum((p1 - p2)**2) 
   
    # initialization algorithm using K-means++
    def initializeCentroid(self, data, k): 
    ## initialize the centroids list and add 
    ## a randomly selected data point to the list 
        centroids = [] 
        ef = np.random.choice(data.shape[0], 1, replace = False)
        #print(ef)
        centroids.append(data[ef[0]]) 
        #plot(data, np.array(centroids)) 
   
    ## compute remaining k - 1 centroids 
        for c_id in range(k - 1):   
        ## initialize a list to store distances of data 
        ## points from nearest centroid 
            dist = [] 
            for i in range(data.shape[0]): 
                point = data[i, :] 
                d = sys.maxsize 
            ## compute distance of 'point' from each of the previously 
            ## selected centroid and store the minimum distance 
                for j in range(len(centroids)): 
                    temp_dist = self.distance(point, centroids[j]) 
                    d = min(d, temp_dist) 
                dist.append(d) 
              
        ## select data point with maximum distance as our next centroid 
            dist = np.array(dist) 
            next_centroid = data[np.argmax(dist), :] 
            centroids.append(next_centroid) 
            dist = [] 
        return centroids 

    def fit(self,data, k):
        np.seterr(divide='ignore', invalid='ignore')
        self.k = k
        self.centroids = {}
        self.classifications = {}
        #ef = np.random.choice(data.shape[0], self.k, replace = False)
        cen = self.initializeCentroid(data,k)
        
        for i in range(self.k):
            self.centroids[i] = cen[i]
        for i in range(self.max_iter):

            for i in range(self.k):
                self.classifications[i] = []
 
            for featureset in data:
                
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                #print(featureset)
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)
                
                #print("----------------")
            prev_centroids = dict(self.centroids)
            #print(self.classifications)
            #print(prev_centroids)
            for classification in self.classifications:
                #print(classification)
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)
                #print(np.average(self.classifications[classification],axis=0))
            #print("++++++++")
            optimized = True

            #print(self.centroids)
            #print("++++++++++++++++")
            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100) > self.tol:
                    
                    #print("----------------------------------")
                    optimized = False

            if optimized:
                break
        self.setClassification(self.classifications)
        
        for c in self.centroids:
            #print(self.centroids[c])
            for a in self.classifications[c]:
                #print((np.linalg.norm(a-c))**2)
                #print(a)
                self.inertia += np.sum((np.linalg.norm(a-self.centroids[c]))**2)
            #print(self.inertia)
            #print("+++++++++++++++++")
        
        self.getLabel(data)
               
    def predict(self,data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification
    
    def getLabel(self, data):
        for row in data:
            self.label.append(self.createLabel(row,self.classify))
    
   